# Historical Crypto Price Analysis

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_style('darkgrid')
sns.set_context('poster', font_scale=1)

from time import sleep, time
from datetime import timedelta, datetime, date, timezone
import requests
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 60

import coinmarketcap
import json

from sklearn.preprocessing import normalize

## CryptoCompare
https://www.cryptocompare.com/api/

In [2]:
def get_all_coin_symbols():
    a = requests.get('https://www.cryptocompare.com/api/data/coinlist/')
    a = a.json()['Data']
    coins = [i for i in a]
    print (len(coins), 'coins on the market, today!')
    return coins

In [3]:
coins = get_all_coin_symbols()

2263 coins on the market, today!


## CoinMarketCap
---
Gets today's top 100 coins in terms of market cap

In [4]:
def get_cmc_data():
    market = coinmarketcap.Market()
    cmc = market.ticker()
    df = pd.DataFrame(cmc)
    df = df[['rank', 'name', 'symbol', 'market_cap_usd', 'price_usd', 'price_btc', 'available_supply', 'max_supply',
             'percent_change_1h', 'percent_change_24h', 'percent_change_7d', '24h_volume_usd']]
    for i in df.columns[3:]:
        df[i] = df[i].astype(float)
    return df

cmc_top100 = get_cmc_data()
top100_sym = list(cmc_top100.symbol)

In [5]:
cmc_top100

,rank,name,symbol,market_cap_usd,price_usd,price_btc,available_supply,max_supply,percent_change_1h,percent_change_24h,percent_change_7d,24h_volume_usd
0,1,Bitcoin,BTC,1.859505e+11,11006.000000,1.000000e+00,1.689538e+07,2.100000e+07,-0.25,2.22,7.16,8.002600e+09
1,2,Ethereum,ETH,8.454763e+10,863.280000,7.869740e-02,9.793767e+07,NaN,-0.20,-0.77,-0.27,1.940290e+09
2,3,Ripple,XRP,3.560731e+10,0.910794,8.303000e-05,3.909480e+10,1.000000e+11,-0.55,-2.51,-8.95,3.587070e+08
3,4,Bitcoin Cash,BCH,2.164896e+10,1273.800000,1.161210e-01,1.699558e+07,2.100000e+07,-0.59,-0.07,-0.39,4.239690e+08
4,5,Litecoin,LTC,1.150465e+10,207.491000,1.891510e-02,5.544648e+07,8.400000e+07,-0.31,-1.52,-0.48,6.547970e+08
5,6,NEO,NEO,8.084375e+09,124.375000,1.133810e-02,6.500000e+07,1.000000e+08,-0.63,-3.63,2.90,1.700240e+08
6,7,Cardano,ADA,7.592639e+09,0.292846,2.670000e-05,2.592707e+10,4.500000e+10,-0.03,-3.58,-14.44,1.210020e+08
7,8,Stellar,XLM,5.994269e+09,0.324574,2.959000e-05,1.846811e+10,NaN,-0.94,-4.77,-14.36,4.109250e+07
8,9,EOS,EOS,5.772153e+09,8.214560,7.488500e-04,7.026735e+08,1.000000e+09,-1.06,-3.18,-4.17,1.925060e+08
9,10,IOTA,MIOTA,5.366606e+09,1.930760,1.760100e-04,2.779530e+09,2.779530e+09,-0.68,-2.07,9.80,2.900130e+07


## CryptoCompare
HistoHour API

In [48]:
current_time = time()
rounded_time = current_time - (current_time % 3600)  # rounded to last hour

In [71]:
def get_hourly_close(fsym, tsym, to_ts):
    if fsym == 'BTC':
        tsym = 'USD'

    df = pd.DataFrame()
    
    before_coin = False
    while before_coin == False:
        r = requests.get('https://min-api.cryptocompare.com/data/histohour?fsym=' + fsym + '&tsym=' + tsym + '&limit=2000&aggregate=1&toTs=' + str(to_ts))
        assert r.status_code == 200
        assert r.json()['Response'] == 'Success'
        sleep(0.25)

        data = r.json()['Data']
        data_list = [(datetime.utcfromtimestamp(j['time']), j['close']) for j in data]
        temp_df = pd.DataFrame(data_list)

        if (temp_df[1].values > 0).sum() != 0:
            df = df.append(temp_df)
        else:
            before_coin = True
        
        to_ts -= 3600*2001
    
    df.columns = ['date', fsym]
    df.date = pd.to_datetime(df.date)
    df.sort_values('date', inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df

In [72]:
def get_top_coins_close(coin_list, coins, to_ts):
    top_coins = pd.DataFrame()
    for first, coin_sym in enumerate(coin_list):
        if coin_sym in coins:
            print (coin_sym)
            df = get_hourly_close(coin_sym, 'BTC', to_ts)
            if first == 0:
                top_coins = df
            else:
                top_coins = top_coins.merge(df, on='date', how='outer')
        else:
            print (coin_sym, '--- not in coins')
    top_coins.sort_values('date', inplace=True)
    top_coins.reset_index(inplace=True, drop=True)
    top_coins.to_csv('../exported_csvs/hourly-prices.csv')
    return top_coins

In [73]:
%time histo_top100 = get_top_coins_close(top100_sym, coins, rounded_time)

BTC
ETH
XRP
BCH
LTC
NEO
ADA
XLM
EOS
MIOTA --- not in coins
DASH
XMR
ETC
XEM
TRX
VEN
USDT
QTUM
LSK
BTG
NANO --- not in coins
OMG
ICX
ZEC
BNB
XVG
BCN
STEEM
DGD
PPT
STRAT
SC
DOGE
RHOC
WAVES
MKR
SNT
BTS
WTC
VERI
ETN
AE
ZRX
DCR
REP
R
KCS
ARDR
GAS
ZCL
KMD
HSR
ARK
BAT
LRC
ZIL
DGB
CNX
DRGN
BTM
SYS
GNT
MONA
DENT
AION
PIVX
POLY
ELF
QASH
RDD
NAS
GBYTE
FCT
KNC
DCN
ETHOS --- not in coins
GXS
BTX
IOST
LINK
SALT
FUN
POWR
XZC
NXT
KIN
PLR
CND
BNT
PART
REQ
ENG
NEBL
SMART
AGI
VTC
PAY
MAID
QSP
WAX
CPU times: user 2min 5s, sys: 2.9 s, total: 2min 8s
Wall time: 14min 12s


In [38]:
print (histo_top100.shape)
histo_top100.head(10)

(40020, 98)


,date,BTC,ETH,XRP,BCH,LTC,ADA,NEO,XLM,EOS,DASH,XMR,ETC,XEM,VEN,TRX,USDT,LSK,BTG,QTUM,OMG,ICX,ZEC,BNB,STEEM,XVG,BCN,DGD,PPT,STRAT,DOGE,SC,RHOC,WAVES,MKR,SNT,BTS,WTC,ZRX,AE,DCR,ETN,REP,VERI,ARDR,ZCL,R,KCS,HSR,KMD,ARK,GAS,BAT,LRC,ZIL,DENT,DGB,BTM,DRGN,MONA,PIVX,SYS,ELF,AION,POLY,RDD,GBYTE,QASH,NAS,GNT,KNC,DCN,BTX,FCT,GXS,LINK,IOST,CNX,FUN,SALT,KIN,XZC,POWR,NXT,PART,REQ,CND,PLR,BNT,ENG,SMART,NEBL,WAX,RDN,AGI,VTC,IGNIS,PAY
0,2013-08-03 07:00:00,100.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-08-03 08:00:00,100.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-08-03 09:00:00,142.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-08-03 10:00:00,142.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-08-03 11:00:00,142.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2013-08-03 12:00:00,113.68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2013-08-03 13:00:00,137.53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2013-08-03 14:00:00,143.65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2013-08-03 15:00:00,114.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [54]:
histo_top100.describe(include='all')

,date,BTC,ETH,XRP,BCH,LTC,ADA,NEO,XLM,EOS,DASH,XMR,XEM,ETC,VEN,TRX,LSK,QTUM,BTG,USDT,OMG,ICX,ZEC,XVG,BCN,STEEM,BNB,PPT,STRAT,SC,RHOC,DOGE,WAVES,SNT,MKR,BTS,AE,R,WTC,ZCL,DCR,ZRX,VERI,REP,DGD,ARDR,HSR,ETN,KMD,KCS,GAS,ARK,DGB,ZIL,BAT,DRGN,SYS,LRC,ELF,POLY,BTM,CNX,MONA,GBYTE,QASH,GNT,AION,PIVX,NAS,KNC,IOST,DCN,RDD,BTX,GXS,FCT,XZC,LINK,POWR,DENT,FUN,NXT,PLR,KIN,SALT,CND,PART,NEBL,ENG,BNT,RDN,VTC,MAID,BLOCK,EMC,REQ,SMART,PAY
count,20010,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,2.001000e+04,2.001000e+04,2.001000e+04,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000
unique,20010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2017-05-31 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2015-11-09 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2018-02-20 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2698.321697,0.037411,0.000035,0.031025,0.009364,0.000004,0.001918,7.671482e-06,0.000141,0.035628,0.011932,2.541375e-05,0.001958,0.000436,4.602774e-07,0.000540,0.000932,0.002905,0.000120,0.000372,0.000053,0.091099,9.970940e-07,2.295467e-07,0.000510,0.000119,0.0

### HistoMinute

In [40]:
def get_histo_min(fsym, tsym, ts):
    if fsym == 'BTC':
        tsym = 'USD'
    to_timestamp = ts  # 12am 02-01-2018 GMT
    df = pd.DataFrame()
    for i in range(1):
        r = requests.get('https://min-api.cryptocompare.com/data/histominute?fsym=' + fsym + '&tsym=' + tsym + '&limit=60&toTs=' + str(to_timestamp))
        assert r.status_code == 200
        assert r.json()['Response'] == 'Success'
        sleep(0.25)

        data = r.json()['Data']
        if len(data) != 0:
            data_list = [(datetime.utcfromtimestamp(j['time']), j['close']) for j in data]
            df = df.append(pd.DataFrame(data_list))
        else:
            break
        
        to_timestamp -= 3600*2001
    
    df.columns = ['date', tsym]
    df.date = pd.to_datetime(df.date)
    df.sort_values('date', inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df

In [41]:
def get_coins_min(coin_list, coins):
    
    # coin_list is list of tuples
    
    top_coins = pd.DataFrame()
    for first, coin_sym in enumerate(coin_list):
        if coin_sym[0] in coins:
            print (coin_sym)
            df = get_histo_min(coin_sym[0], coin_sym[1], 1519097400)
            if first == 0:
                top_coins = df
            else:
                top_coins = top_coins.merge(df, on='date', how='outer')
        else:
            print (coin_sym, '--- not in coins')
    top_coins.sort_values('date', inplace=True)
    top_coins.reset_index(inplace=True, drop=True)
    return top_coins

In [42]:
etc_prices = [('ETC', 'ETH'), ('ETC', 'BTC'), ('ETC', 'USD')]

In [43]:
etc = get_coins_min(test, coins)

('ETC', 'ETH')
('ETC', 'BTC')
('ETC', 'USD')


In [45]:
etc[(etc.ETH >= 0.0446) & (etc.ETH <= 0.0447)]

,date,ETH,BTC,USD
1,2018-02-20 02:31:00,0.04468,0.003691,42.23
4,2018-02-20 02:34:00,0.04466,0.003695,42.27
5,2018-02-20 02:35:00,0.04470,0.003707,42.23
23,2018-02-20 02:53:00,0.04470,0.003713,42.30
41,2018-02-20 03:11:00,0.04467,0.003690,41.97
55,2018-02-20 03:25:00,0.04464,0.003701,42.08


## Volume